<a href="https://colab.research.google.com/github/esiege/BoatCounterRCNN/blob/main/odfw_boat_counter_ai535.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## downloads / drive


In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git
!git clone https://github.com/philferriere/cocoapi.git
%cd ~/../content/Mask_RCNN
!gdown --id --continue 1QHsSNZN-kaOZp8x3aRhIhDIwQ14tHUVz #mask_rcnn_coco.h5 hosted on Jesse's drive
!python3 setup.py install
%cd ~/../content/
!gdown --folder --id --continue 1V8wN7CGQKJj9cKEqXboSr8ICcnedfg55 #persistant folder on Jesse's drive
!gdown --folder --id --continue 1TFSdI70fR-EGZ_sxUpkI6Sl7hMImhhzQ #models folder on Jesse's drive
!gdown --id --continue 18L0ucK4YyoJsPg5seJEodygWrzWqTEU5 #frames zip from Jesse's drive
!unzip frames.zip
%cd ~/../content/

Streaming output truncated to the last 5000 lines.
  inflating: Frames/frame3046.jpg    
  inflating: Frames/frame3047.jpg    
  inflating: Frames/frame3048.jpg    
  inflating: Frames/frame3049.jpg    
  inflating: Frames/frame305.jpg     
  inflating: Frames/frame3050.jpg    
  inflating: Frames/frame3051.jpg    
  inflating: Frames/frame3052.jpg    
  inflating: Frames/frame3053.jpg    
  inflating: Frames/frame3054.jpg    
  inflating: Frames/frame3055.jpg    
  inflating: Frames/frame3056.jpg    
  inflating: Frames/frame3057.jpg    
  inflating: Frames/frame3058.jpg    
  inflating: Frames/frame3059.jpg    
  inflating: Frames/frame306.jpg     
  inflating: Frames/frame3060.jpg    
  inflating: Frames/frame3061.jpg    
  inflating: Frames/frame3062.jpg    
  inflating: Frames/frame3063.jpg    
  inflating: Frames/frame3064.jpg    
  inflating: Frames/frame3065.jpg    
  inflating: Frames/frame3066.jpg    
  inflating: Frames/frame3067.jpg    
  inflating: Frames/frame3068.jpg    

In [ ]:
%mkdir InstanceMasks
%mkdir InstanceMasks/boat
%mkdir InstanceMasks/person
%mkdir InstanceSnips
%mkdir InstanceSnips/boat
%mkdir InstanceSnips/person
%mkdir InstanceSilhouette
%mkdir InstanceSilhouette/boat
%mkdir InstanceSilhouette/person

In [ ]:
!pip install tensorflow==1.14.0
!pip install keras==2.2.5
!pip install numpy==1.21.6
!pip install scipy==1.4.1
!pip install Pillow==7.1.2
!pip install cython==0.29.30
!pip install matplotlib==3.2.2
!pip install scikit-image==0.18.3
!pip install opencv-python==4.1.2.30
!pip install h5py==2.10.0
!pip install imgaug==0.2.9
!pip install IPython[all]
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
#!pip install image-similarity-measures
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install opencv-python-headless==4.5.4.60 fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 1.1 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 488 kB 51.4 MB/s 
     |████████████████████████████████| 3.1 MB 55.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
    Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636
ERROR: pip's dependency resolver does not currently t

## Imports

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
#from google.colab.patches import cv2_imshow
import shutil
import fiftyone as fo
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torchvision import datasets, models, transforms
from torchvision.transforms import functional as func
import torch.nn as nn
import torch.nn.functional as F

# Root directory of the project
ROOT_DIR = os.path.abspath("./Mask_RCNN/")
print(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # find local version
import coco

%matplotlib inline

NumExpr defaulting to 4 threads.
Migrating database to v0.16.2
/content/Mask_RCNN


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("CUDA: " + str(torch.cuda.is_available()))

CUDA: True


##Models

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

class InferenceConfig(coco.CocoConfig):
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

# MaskRCNN model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

##  COCO Class Names
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:2139: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:2239: The name tf.image.resize_nearest_neighbor is d

In [ ]:
#Our transfer training model, which identifies commercial or recreational boats

#https://colab.research.google.com/drive/1ivCSaNDdKTI-KRkr5Fy5koxoBkQfRi_h

model_ft = models.resnet152(pretrained=True)
model_ft.fc = nn.Linear(model_ft.fc.in_features, 2)
model_ft.load_state_dict(torch.load("/content/Models/Recreational v Commercial Boats Model_state_dict"))
model_ft_labels = ["Commercial", "Recreational"]
model_ft.eval()

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## RCNN - Generate Masks

In [ ]:
def mask(filename, show_images=True):
  image = skimage.io.imread(filename)

  # Run detection
  results = model.detect([image], verbose=0)[0]

  # Visualize results
  if show_images == True:
    visualize.display_instances(image, results['rois'], results['masks'], results['class_ids'], class_names, results['scores'],figsize=(25,25))

  #print(results)
  return results, image

In [ ]:
# classifier for MaskRCNN.  We iterate through a the 'frames' directory downloaded through google drive

train_frames = list(range(540,567)) # frames 540 through 567 is an interesting segment
show_images = True #print images below with masking

#for loading runs
load_downloaded_mask = False
downloaded_mask_path = '/content/Persistant/mask_results (530-600).npy'

IMAGE_WIDTH = 1152
IMAGE_HEIGHT = 720

def classifyFrames():
  results = []

  with fo.ProgressBar() as pb:
    for i in pb(train_frames):
      dir = "Frames/"
      filename = "frame" + str(i)
      ext = ".jpg"
      path = dir + filename + ext
      mask_result, image = mask(path, show_images)
      mask_result["image"] = image
      mask_result["filename"] = filename
      mask_result["path"] = path
      results.append(mask_result)

    np.save('/content/mask_results.npy', results)
    return results

#for loading runs
if load_downloaded_mask == True:
  mask_results = np.load(downloaded_mask_path, allow_pickle=True)
else:
  mask_results = classifyFrames()

In [ ]:
#fifty one dataset, used for visualizer
fo_dataset = fo.Dataset("fo_dataset")
with fo.ProgressBar() as pb:
  for i in pb(train_frames):
    dir = "Frames/"
    filename = "frame" + str(i)
    ext = ".jpg"
    path = dir + filename + ext
    fo_dataset.add_sample(fo.Sample(filepath=path))
fo_dataset = fo.load_dataset("fo_dataset")
fo_dataset.persistent = True

 100% |███████████████████| 27/27 [50.0ms elapsed, 0s remaining, 539.5 samples/s] 


## Main

In [ ]:
def printBoatFrames():
  totalCount = 0
  for i,frame in enumerate(frames):
    print("-----" + str(i) + "----------------------------------------------------------------------------------------------------" )
    for j,boat in enumerate(frame):
      print("pos: " + str("{:10.2f}".format(boat.xCenterN)) + "  -  " +
            "frame: " + str(boat.filename) + "  -  " +
            "inst: " + str(boat.instanceId) + "  -  " +
            "type: " + str(boat.getBoatTypeName()) + "  -  " +
            "model_results: " + str(boat.model_results) + "  -  " +
            "status: " + str(boat.status).ljust(12, ' ') + "  -  " +
            "speed: " + str("{:10.2f}".format(boat.speed) + "  -  " +
            "fCnt: " + str(boat.frameCount))
            )
      if boat.status == "Leaving (-1)":
          totalCount = totalCount - 1
      if boat.status == "Leaving (+1)":
          totalCount = totalCount + 1

  print("===============================")
  print("Total Count: " + str(totalCount))

In [ ]:
# our main building block - the boat frame.  A boat frame represents a single bounding box in a single image frame.
# we store all boat details within this class, along with a reference to the previous BoatFrame

class BoatFrame():
  def __init__(self, filename, path, score, image, imageIndex, mask, roi):

    #inits
    self.filename = filename        # filename on disk
    self.path = path                # filepath on disk
    self.score = score              # RCNN score
    self.image = image              # the image data for the entire frame
    self.imageIndex = imageIndex    # the RCNN image index
    self.mask = mask                # the masking data from RCNN
    self.roi = roi                  # the bounding box coords from RCNN, stored as [y1, x1, y2, x2]

    #calculated
    self.xCenter = (self.roi[1] + self.roi[3])/2  # to determine boat position, we mainly use the centroid of the x coordinate,
                                                  # which allows us to simply compare scalars for movement calculations
    self.xCenterN = self.xCenter / IMAGE_WIDTH    # this is normalized by the image width to get a value 0 > x > 1
    self.bounded_image = self.getBoundedImage()   # the data from the bounding box over the image, used for rec/commercial classification
    self.model_results = self.getModelResults()   # the results from running the transfer learning model
    self.boatTypeId = self.getBoatTypeId()        # the boat type from the transfer learning model

    #set later
    self.status = "Unknown"     # status is the current action of the boat during this frame.  A boat starts as 'Entering', then is
                                # 'Moving' as it traverses the frame, and finally leaves the frame as 'Leaving,' resulting in a count value (bi-directional)
    self.frameCount = 0         # the amount of frames that the boat instance exists for
    self.prevFrame = None       # a ref to the previous frame, so we can traverse back the list
    self.instanceId = None      # the unique id of this boat to distinguish it from all other boats
    self.speed = 0              # the speed of the boat, based on normalized centroid distance between frames

  def getOriginalInstance(self):    # a method to traverse prevFrames and find the boat 'Entering' aka the head
    b = self.prevFrame
    while b.prevFrame is not None:
      b = b.prevFrame
    return b

  def getBoundedImage(self):        # a method to get image data from the bounding box area
    y1, x1, y2, x2 = self.roi
    bounded_image = self.image[y1:y2, x1:x2]
    #cv2_imshow(bounded_image) #print images
    return bounded_image

  def getModelResults(self):              # a method to run the transfer learning model we have preloaded
    im = self.bounded_image
    im = func.to_tensor(im)#.to(device)
    im = torch.unsqueeze(im,0)
    p, i = torch.max(model_ft(im), 1)
    return model_ft(im)

  def getBoatTypeId(self):                # get rec or commercial boat type id from model results
    p, i = torch.max(self.model_results, 1)
    return i

  def getBoatTypeName(self):              #get label from model result id
    return model_ft_labels[self.boatTypeId]

In [ ]:
# now that we have defined a nice looking class for our boat frames, it's time to cram the MRCNN result data into it

def initBoatFrames():
  frames = []
  for index, element in enumerate(mask_results): #mask_results is the MRCNN result data
      boatFrames = []
      imageIndex = 0
      for i, classId in enumerate(mask_results[index]["class_ids"]):
        if class_names[classId] == "boat":
          boatFrame = BoatFrame(
              mask_results[index]["filename"],
              mask_results[index]["path"],
              mask_results[index]["scores"][i],
              mask_results[index]["image"],
              imageIndex,
              mask_results[index]["masks"][i],
              mask_results[index]["rois"][i]
              )
          imageIndex = imageIndex + 1
          boatFrames.append(boatFrame)
      frames.append(boatFrames)
  return frames

frames = initBoatFrames()

In [ ]:
# now we can process masking and instance data that we have collected.  MASK CRNN is great in that it retains the instance identification tracking in the form
# of an ordered list, so we know that if we have the same boats on a different frame, the order and thus id will be preserved.  However, this does not solve
# all instance tracking related issues.  If a boat leaves, it is not obvious which item from your ordered list has gone missing.  If a boat enters at the same time,
# our process needs to be smart enough to re-assign a new instance identification.  Thus we have a processor method:

def processBoatFrames():

  # some variables for determining various cutoff points when making decisions about new instances:
  newInstanceThreshhold = 0.25    # when a new boat enters, ignore if its inside this offest
  movementSideThreshhold = 0.01   # when deciding if a boat has moved off-frame; speed + pos + this > 1 or < 0
  minFrameCountThreshhold = 3     # if a boat leaves with less frames on screen than this, ignore it

  def setEntering(b, newInstanceId):      # a method for setting a boatframe to it's proper values when it has the 'Entering' status
    b.status = "Entering"
    b.frameCount = 1
    b.instanceId = newInstanceId
    return newInstanceId + 1

  def setMoving(b, prev_b):               # a method for setting a boatframe to it's proper values when it has the 'Moving' status
    b.status = "Moving"
    b.frameCount = prev_b.frameCount + 1
    b.speed = b.xCenterN - prev_b.xCenterN
    b.instanceId = prev_b.instanceId
    b.prevFrame = prev_b

  newInstanceId = 0

  for i,frame in enumerate(frames):           # We enumerate 'frames,' which is our list of strongly typed BoatFrames

    if i == 0:                                # Hold on to a previous frame, if it exists
      prevFrame = None
      boatCntChange = 0
    else:
      prevFrame = frames[i-1]
      boatCntChange = len(frame) - len(prevFrame)

    for j,boat in enumerate(frame):                         # We now enumerate the boats inside this frame

      if len(prevFrame) == 0:                               # Somewhat of a base case - if we have no boats on the previous frame, set this boat to 'Entering'
        newInstanceId = setEntering(boat, newInstanceId)

      if prevFrame != None:
        if boatCntChange >= 0 and j < len(prevFrame):       # If we are in the intersection of a previous frame and a new frame, we can call those boats 'Moving'
          setMoving(boat, prevFrame[j])

        if boatCntChange >= 0 and j >= len(prevFrame):      # If we are outside the intersection, and have new boats to deal with, we can call those 'Entering'

          # EDGE CASE protection 1 - check if the new instance is in the center of the screen (false positives).  If so, remove it
          if boat.xCenterN > newInstanceThreshhold and boat.xCenterN < (1-newInstanceThreshhold):
            frame.remove(boat)
            continue

          newInstanceId = setEntering(boat, newInstanceId)


        if boatCntChange < 0:                               # If the frame has less boats than the previous frame, we need to figure out what boat left, and to count which direction it went
          for k, prevBoat in enumerate(prevFrame):

            movedOffscreenRight = prevBoat.xCenterN + prevBoat.speed > 1-movementSideThreshhold   # we can use our x-centroids and our boat speed to determine which side of the screen it went to
            movedOffscreenLeft = prevBoat.xCenterN + prevBoat.speed < 0+movementSideThreshhold

            if movedOffscreenRight or movedOffscreenLeft:
              # EDGE CASE protection 2 - check if the boat leaving has not reached the threshold for its frame count.  If not, we arent counting it.  Remove it instead.
              if prevBoat.frameCount < minFrameCountThreshhold:
                prevFrame.remove(prevBoat)
                continue
              if movedOffscreenRight and prevBoat.getOriginalInstance().xCenterN < 0.5:
                prevBoat.status = "Leaving (-1)"
                continue
              if movedOffscreenLeft and prevBoat.getOriginalInstance().xCenterN > 0.5:
                prevBoat.status = "Leaving (+1)"
                continue
              prevBoat.status = "Leaving (No Count)" #boat turned around!

            else:
              setMoving(boat, prevBoat)    # other boats in the frame are moving


              # EDGE CASE protection 3 - remove new boats that are only in 1 frame and are not close to their following frame.
              # this edge case happens when a false positive is detected on the same frame that a boat leaves AND a boat enters the frame. whew
              if k > j:
                b_diff = 0
                b_boat = 0
                for eb in prevFrame[len(frame):len(prevFrame)]: # iterate additional boats in prev frame
                  diff = abs(eb.xCenterN - boat.xCenterN)
                  if diff > b_diff and eb.status[0:7] is not "Leaving":
                    b_diff = diff
                    b_boat = eb
                prevFrame.remove(b_boat)


    if prevFrame and boatCntChange < 0 and len(frame) == 0: # finally, if there are no boats on the current frame, handle them in a similar manner to above
      for k, prevBoat in enumerate(prevFrame):

        movedOffscreenRight = prevBoat.xCenterN + prevBoat.speed > 1-movementSideThreshhold
        movedOffscreenLeft = prevBoat.xCenterN + prevBoat.speed < 0+movementSideThreshhold

        if movedOffscreenRight or movedOffscreenLeft:
          if prevBoat.frameCount < minFrameCountThreshhold:
            prevFrame.remove(prevBoat)
            continue
          if movedOffscreenRight and prevBoat.getOriginalInstance().xCenterN < 0.5:
            prevBoat.status = "Leaving (-1)"
            continue
          if movedOffscreenLeft and prevBoat.getOriginalInstance().xCenterN > 0.5:
            prevBoat.status = "Leaving (+1)"
            continue
          prevBoat.status = "Leaving (No Count)" #boat turned around


  # We perform a quick 2nd pass to count the number of class labels for each instance, and use the max across all instance frames.  This just increases
  # the classification accuracy and consisitancy across all frames of the insteance
  for i,frame in reversed(list(enumerate(frames))):
    for j,boat in reversed(list(enumerate(frame))):
      p = boat.prevFrame
      bt = []
      while p is not None:
        bt.append(p.boatTypeId)
        p = p.prevFrame
      if len(bt) > 0:
        boat.boatTypeId = max(bt,key=bt.count)


processBoatFrames()
printBoatFrames()

-----0----------------------------------------------------------------------------------------------------
-----1----------------------------------------------------------------------------------------------------
pos:       0.99  -  frame: frame541  -  inst: 1  -  type: Commercial  -  model_results: tensor([[ 1.8347, -1.1847]], grad_fn=<AddmmBackward0>)  -  status: Entering      -  speed:       0.00  -  fCnt: 1
-----2----------------------------------------------------------------------------------------------------
pos:       0.96  -  frame: frame542  -  inst: 1  -  type: Commercial  -  model_results: tensor([[ 0.1803, -0.0134]], grad_fn=<AddmmBackward0>)  -  status: Moving        -  speed:      -0.03  -  fCnt: 2
pos:       0.98  -  frame: frame542  -  inst: 2  -  type: Commercial  -  model_results: tensor([[ 0.9780, -1.1317]], grad_fn=<AddmmBackward0>)  -  status: Entering      -  speed:       0.00  -  fCnt: 1
-----3-------------------------------------------------------------------

## Visualizer

In [ ]:
#viola!  We plug in the fancy fifty one visualizer to examin our results!

def visualizeResults():
  totalCountIn = 0
  totalCountOut = 0
  for i,sample in enumerate(fo_dataset):
    frame = frames[i]
    detections = []
    for j,boat in enumerate(frame):

      y1, x1, y2, x2 = boat.roi
      rel_box = [x1 / IMAGE_WIDTH, y1 / IMAGE_HEIGHT, (x2 - x1) / IMAGE_WIDTH, (y2 - y1) / IMAGE_HEIGHT]

      detections.append(
          fo.Detection(
              label=str(boat.instanceId) + ": " + str(boat.getBoatTypeName()),
              bounding_box=rel_box,
              confidence=1,
              mask=boat.mask
          )
      )

      if boat.status == "Leaving (-1)":
          totalCountOut = totalCountOut + 1
      if boat.status == "Leaving (+1)":
          totalCountIn = totalCountIn + 1

    if len(detections) > 0:
      sample["instances"] = fo.Detections(detections=detections)
      sample.save()


  print("===============================")
  print("Total Count In: " + str(totalCountIn))
  print("Total Count Out: " + str(totalCountOut))
  print("===============================")

  fo_dataset.draw_labels("fo_output")
  session = fo.launch_app(fo_dataset)

visualizeResults()

Total Count In: 3
Total Count Out: 1
 100% |███████████████████| 27/27 [1.5s elapsed, 0s remaining, 18.2 samples/s]         
